In [114]:
%matplotlib inline
import pandas as pd
import matplotlib
import time
import numpy as np
import matplotlib.pyplot as plt
TRADE_DATA = "../data/Trade.csv"

In [123]:
df_trade = pd.read_csv(TRADE_DATA)
df_trade.columns
df_trade.describe()

,TradeDateKey,CustomerIdx,IsinIdx,NotionalEUR,Price,CustomerInterest
count,6.762021e+06,6.762021e+06,6.762021e+06,6.762021e+06,2.144088e+06,6.762021e+06
mean,2.016750e+07,1.922354e+03,1.460369e+04,6.313228e+06,1.779008e+05,3.271106e-01
std,6.768082e+03,8.579293e+02,7.963809e+03,2.712408e+08,1.318503e+06,4.691581e-01
min,2.016010e+07,0.000000e+00,0.000000e+00,-1.485547e+08,-9.999990e+05,0.000000e+00
25%,2.016063e+07,1.288000e+03,7.392000e+03,3.410980e+05,9.313780e+01,0.000000e+00
50%,2.017023e+07,2.090000e+03,1.522900e+04,8.812120e+05,1.010000e+02,0.000000e+00
75%,2.017093e+07,2.574000e+03,2.211900e+04,2.136842e+06,1.075000e+02,1.000000e+00
max,2.018042e+07,3.470000e+03,2.739400e+04,2.000000e+11,9.999999e+06,1.000000e+00


In [125]:
df_trade.sort_values("TradeDateKey", ascending=True).tail(100)


,TradeDateKey,CustomerIdx,IsinIdx,BuySell,NotionalEUR,Price,TradeStatus,CustomerInterest
395644,20180420,2743,14216,Buy,875415.0,98.535,Done,1.0
1280173,20180420,2195,24062,Buy,1335000.0,101.232,Done,1.0
5675819,20180422,3032,6499,Buy,817906.0,NaN,Holding,0.0
5678090,20180422,3032,15303,Buy,1615037.0,NaN,Holding,0.0
5678094,20180422,3032,15308,Buy,2658687.0,NaN,Holding,0.0


In [116]:
# Filter the data
df_trade = df_trade[df_trade["TradeDateKey"] >= 20171008]
# Delete Holding Values
df_trade = df_trade[df_trade["TradeStatus"] != "Holding"]

# Drop Useless Columns
df_trade = df_trade.drop(["TradeStatus", "NotionalEUR", "Price"], axis=1)
df_trade = df_trade.sort_values("TradeDateKey", ascending=True)

In [121]:
# Positive Samples
positive_samples = df_trade[(df_trade["TradeDateKey"] >= 20180328) & (df_trade["TradeDateKey"] < 20180405)]
# positive_samples = positive_samples.groupby(["CustomerIdx", "IsinIdx"]).count()["CustomerInterest"].reset_index(level=['CustomerIdx', 'IsinIdx'])
# positive_samples["CustomerInterest"] = positive_samples["CustomerInterest"].apply(lambda x: 0 if x > 1 else x)

# Negative Samples
negative_samples = df_trade
# negative_samples = negative_samples.groupby(["CustomerIdx", "IsinIdx"]).count()["CustomerInterest"].reset_index(level=['CustomerIdx', 'IsinIdx'])

# negative_samples["CustomerInterest"] = negative_samples["CustomerInterest"].apply(lambda x: 0 if x > 0 else x)
negative_samples = negative_samples.groupby(["CustomerIdx", "IsinIdx"]).count()
negative_samples = negative_samples[negative_samples["CustomerInterest"] <= 2]
negative_samples["CustomerInterest"] = negative_samples["CustomerInterest"].apply(lambda x: 0 if x > 0 else x)
print(negative_samples.describe())
print(negative_samples.head())
negative_samples = negative_samples.reset_index(level=['CustomerIdx', 'IsinIdx'])
print(negative_samples.describe())
print(negative_samples.head())


# print( # ["CustomerInterest"].apply(lambda x: )
# negative_samples = negative_samples.sort_values("CustomerIdx", ascending=True)

test_set = pd.concat([positive_samples, negative_samples])
# test_set = test_set.sort_values("CustomerIdx", ascending=True)
# test_set.groupby(["CustomerIdx", "IsinIdx"]).sum().reset_index(level=['CustomerIdx', 'IsinIdx'])
print(test_set.describe())
test_set.sort_values("CustomerIdx", ascending=True).head(10)

        TradeDateKey        BuySell  CustomerInterest
count  219606.000000  219606.000000          219606.0
mean        1.233596       1.233596               0.0
std         0.423119       0.423119               0.0
min         1.000000       1.000000               0.0
25%         1.000000       1.000000               0.0
50%         1.000000       1.000000               0.0
75%         1.000000       1.000000               0.0
max         2.000000       2.000000               0.0
                     TradeDateKey  BuySell  CustomerInterest
CustomerIdx IsinIdx                                         
0           21856               2        2                 0
            24944               1        1                 0
            25992               2        2                 0
            26726               1        1                 0
            27045               2        2                 0
         CustomerIdx        IsinIdx   TradeDateKey        BuySell  \
count  219606.0000

,BuySell,CustomerIdx,CustomerInterest,IsinIdx,TradeDateKey
4,2,0,0,27045,2
0,2,0,0,21856,2
1,1,0,0,24944,1
2,2,0,0,25992,2
3,1,0,0,26726,1
5,1,0,0,27086,1
6,1,0,0,27298,1
11,1,1,0,16400,1
10,1,1,0,10244,1
9,1,1,0,10243,1


In [40]:
df_trade.head()

TradeDateKey  BuySell  CustomerInterest  CustomerIdx
CustomerIdx IsinIdx                                                      
2           5215                2        2                 2            1
            16812               2        2                 2            1
            17091               1        1                 1            1
            17275               2        2                 2            1
14          22209               2        2                 2            1

In [11]:
df_trade.groupby("TradeDateKey").count()["CustomerInterest"].values

array([     1,   3524,   5317,   4808,   5031,      2,     19,   5549,
         4622,   5779,   5827,   5731,      5,   1951,   4172,   5096,
         6523,   4711,      7,   4919,   5332,   4708,   6833,   4561,
           15,   5881,   7591, 197510,   6383,   5156,      5,   6088,
         6513,   6159,   7461,   6590,      2,   6710,   6266,   5346,
         5850,   4414,     10,   1618,   5162,   5078,   7130,   4732,
            2,      6,   7591,   5091, 120770,   5307,   4602,   4970,
         5240,   4535,   4108,   3815,   5219,   4615,   4993,   5917,
         4180,   6051,  10461,   6197,   5355,   1878,      1,      1,
         6989,   5462,   6153,   9472,     97, 117437,   1793,   5577,
         4742,  10166,   5414,      1,     14,   3949,   4809,   6407,
         5500,   5401,     49,   4722,   5455,   5987,   5140,   4768,
            3])

In [8]:
dictionary = df_trade.groupby(['CustomerIdx', 'IsinIdx']).count()['TradeDateKey'].to_dict()
dictionary[(0, 24944)]

1

In [41]:
df_trade.describe()
print(np.sort(df_trade["TradeDateKey"].unique()))

TEST_DATE = 20180412
VAL_DATE = 20180405
TRAIN_DATE = 20180328

[20180101 20180102 20180103 20180104 20180105 20180106 20180107 20180108
 20180109 20180110 20180111 20180112 20180114 20180115 20180116 20180117
 20180118 20180119 20180121 20180122 20180123 20180124 20180125 20180126
 20180128 20180129 20180130 20180131 20180201 20180202 20180204 20180205
 20180206 20180207 20180208 20180209 20180211 20180212 20180213 20180214
 20180215 20180216 20180218 20180219 20180220 20180221 20180222 20180223
 20180224 20180225 20180226 20180227 20180228 20180301 20180302 20180305
 20180306 20180307 20180308 20180309 20180312 20180313 20180314 20180315
 20180316 20180319 20180320 20180321 20180322 20180323 20180324 20180325
 20180326 20180327 20180328 20180329 20180330 20180331 20180402 20180403
 20180404 20180405 20180406 20180407 20180408 20180409 20180410 20180411
 20180412 20180413 20180415 20180416 20180417 20180418 20180419 20180420
 20180422]


In [59]:
# Create a Double KEY dictionary
"""train = df_trade.values

from collections import defaultdict
from itertools import product

customer_ids = np.unique(train[:,1])
bonds_ids = np.unique(train[:,2])

cbidx_freq = defaultdict(dict)
for c, b in product(customer_ids, bonds_ids):
    for sample in train:
        i = 0
        if sample[1] == c and sample[2] == b:
            i = i + 1
        cbidx_freq[c][b] = i"""
"""max_value = df_trade["TradeDateKey"].max()
df_trade[df_trade["TradeDateKey"] == max_value].groupby("CustomerIdx").count()["CustomerInterest"]"""


# Generate the label
df_trade.groupby(['CustomerIdx','IsinIdx']).count()['TradeDateKey']

# Generate the train set
df_trade[(df_trade["TradeDateKey"] >= TRAIN_DATE) & (df_trade["TradeDateKey"] < VAL_DATE)].groupby(["CustomerIdx", "IsinIdx"]).count().describe()
df_trade[(df_trade["TradeDateKey"] >= VAL_DATE) & (df_trade["TradeDateKey"] < TEST_DATE)].groupby(["CustomerIdx", "IsinIdx"]).count().describe()
df = df_trade[df_trade["TradeDateKey"] >= TEST_DATE].groupby(["CustomerIdx", "IsinIdx"]).count()["CustomerInterest"]

print(df.head())
print(df.values[0:5])
df.reset_index(level=['CustomerIdx', 'IsinIdx'])

CustomerIdx  IsinIdx
0            27045      1
             27298      1
5            10842      1
8            1812       1
             3364       1
Name: CustomerInterest, dtype: int64
[1 1 1 1 1]


,CustomerIdx,IsinIdx,CustomerInterest
0,0,27045,1
1,0,27298,1
2,5,10842,1
3,8,1812,1
4,8,3364,1
5,8,21277,1
6,8,24116,1
7,8,25460,1
8,8,26341,1
9,9,7504,1


In [6]:
from itertools import islice
dictionary = df_trade.groupby('IsinIdx').count()['TradeDateKey'].to_dict()

def take(n, iterable):
    return list(islice(iterable, n))
def print_dictionary(dictionary=None):
    n_items = take(10, dictionary.items())
    for key, val in n_items:
        print(key, val)
    return
print_dictionary(dictionary)

1 7
2 10
3 1
4 4
5 1
6 40
7 76
14 2
15 8
17 31
